In [8]:
import concurrent.futures
import json
import os
import re
from typing import List, Tuple
from datasets import Dataset
from openai import OpenAI
from pydantic import Field
from tqdm.auto import tqdm
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
def load_articles_from_json(file_path: str) -> Dataset:
    with open(file_path, "r") as file:
        data = json.load(file)
    return Dataset.from_dict(
        {
            "id": [item["id"] for item in data["artifact_data"]],
            "content": [item["content"] for item in data["artifact_data"]],
            "platform": [item["platform"] for item in data["artifact_data"]],
            "author_id": [item["author_id"] for item in data["artifact_data"]],
            "author_full_name": [item["author_full_name"] for item in data["artifact_data"]],
            "link": [item["link"] for item in data["artifact_data"]],
        }
    )

def clean_text(text):
    # Remove non-alphanumeric characters except for apostrophe, periods, commas, exclamation marks and question marks
    text = re.sub(r"[^\w\s.,!?']", " ", text)
    # Replace multiple consecutive whitespace characters with a single space
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def extract_substrings(dataset: Dataset, min_length: int = 1000, max_length: int = 2000) -> List[str]:
    extracts = []

    # To make sure that the splitting doesn't break sentences, which could modify their meanings
    # Use a regex to only split after the end of a sentence.
    sentence_pattern = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s"
    for article in dataset["content"]:
        cleaned_article = clean_text(article)
        sentences = re.split(sentence_pattern, cleaned_article)

        current_chunk = ""
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            if len(current_chunk) + len(sentence) <= max_length:
                current_chunk += sentence + " "
            else:
                if len(current_chunk) >= min_length:
                    extracts.append(current_chunk.strip())
                current_chunk = sentence + " "

        if len(current_chunk) >= min_length:
            extracts.append(current_chunk.strip())

    return extracts


In [10]:
class PreferenceSet:
    def __init__(self, triples: List[Tuple[str, str, str]]):
        # instructions, generated answers(rejected), and extracted answers(chosen)
        self.triples = triples

    @classmethod
    def from_json(cls, json_str: str) -> "PreferenceSet":
        data = json.loads(json_str)
        triples = [
            (
                triple["instruction"],
                triple["generated_answer"],
                triple["extracted_answer"],
            )
            for triple in data["preference_triples"]
        ]
        return cls(triples)

    def __iter__(self):
        return iter(self.triples)

In [11]:
def generate_preference_triples(
    extract: str, client: OpenAI
) -> List[Tuple[str, str, str]]:
    prompt = f"""Based on the following extract, generate five instruction-answer triples.
Each triple should consist of:
1. An instruction asking about a specific topic in the context.
2. A generated answer that attempts to answer the instruction based on the context.
3. An extracted answer that is a relevant excerpt directly from the given context.

Instructions must be self-contained and general, without explicitly mentioning a context, system, course, or extract.

Important:
- Ensure that the extracted answer is a verbatim copy from the context, including all punctuation and apostrophes.
- Do not add any ellipsis (...) or [...] to indicate skipped text in the extracted answer.
- If the relevant text is not continuous, use two separate sentences from the context instead of skipping text.

Provide your response in JSON format with the following structure:
{{
    "preference_triples": [
        {{
            "instruction": "...",
            "generated_answer": "...",
            "extracted_answer": "..."
        }},
    ...
    ]
}}

    Extract:
    {extract}
"""
    # Specify a system prompt to guide the model into generating triples
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant who generates instruction-answer triples based on the given context. \
                    Each triple should include an instruction, a generated answer, and an extracted answer from the context. \
                    Provide your response in JSON format.",
            },
            {"role": "user", "content": prompt},
        ],
        response_format={"type": "json_object"},
        max_tokens=2000,
        temperature=0.7,
    )
    # Parse the structured output
    result = PreferenceSet.from_json(completion.choices[0].message.content)
    # Convert to list of tuples
    return result.triples

In [12]:
def filter_short_answers(dataset: Dataset, min_length: int = 100) -> Dataset:
    """Filter out short answers from the dataset based on a minimum length threshold."""

    def is_long_enough(example):
        return len(example["chosen"]) >= min_length

    filtered_dataset = dataset.filter(is_long_enough)
    return filtered_dataset


def filter_answer_format(dataset: Dataset) -> Dataset:
    """Filter out the answers start with an uppercase letter and end with proper punctuation."""

    def is_valid_format(example):
        chosen = example["chosen"]
        return len(chosen) > 0 and chosen[0].isupper() and chosen[-1] in {".", "!", "?"}

    filtered_dataset = dataset.filter(is_valid_format)
    return filtered_dataset

In [13]:
def create_preference_dataset(dataset: Dataset, client: OpenAI, num_workers=4) -> Dataset:
    extracts = extract_substrings(dataset)
    preference_triples = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [
            executor.submit(generate_preference_triples, extract, client)
            for extract in extracts
        ]
        for future in tqdm(
            concurrent.futures.as_completed(futures), total=len(futures)
        ):
            preference_triples.extend(future.result())
    
    instructions, generated_answers, extracted_answers = zip(*preference_triples)
    return Dataset.from_dict({
        "prompt": list(instructions),
        "rejected": list(generated_answers),
        "chosen": list(extracted_answers)
    })


In [ ]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# 1. Load the raw data
raw_dataset = load_articles_from_json("../output/cleaned_documents.json")
print("Raw dataset:")
print(raw_dataset.to_pandas())

# 2. Create preference dataset
dataset = create_preference_dataset(raw_dataset, client)
print("Preference dataset:")
print(dataset.to_pandas())

# 3. Filter out samples with short answers
dataset = filter_short_answers(dataset)

# 4. Filter answers based on format
dataset = filter_answer_format(dataset)

# 5. Export
dataset.push_to_hub("michaelnguyen11/llm_twin_dpo")


Raw dataset:
                                     id  \
0  4eb69ac6-f8d3-4766-8e49-0246f9d2e9a8   

                                             content                 platform  \
0  Building ML System Using the FTI Architecture ...  decodingml.substack.com   

                              author_id author_full_name  \
0  c7167a83-e446-428d-9a2e-6e88a6b8bc6f      Hiep Nguyen   

                                                link  
0  https://decodingml.substack.com/p/building-ml-...  


  0%|          | 0/7 [00:00<?, ?it/s]

Preference dataset:
                                               prompt  \
0   What is the main challenge related to client a...   
1   What is the FTI architecture designed to address?   
2   What are the three critical steps that any ML ...   
3   How does the FTI pattern provide clarity in an...   
4   What is a significant aspect of the design fle...   
5   What are some responsibilities of an ML or MLO...   
6   What components are necessary for a mature ML ...   
7   How can all components of an ML system be conn...   
8   What is the typical architecture present in mo...   
9   What is the first step in building scalable ML...   
10  What is the FTI architecture used for in ML sy...   
11  What are the initial steps discussed in buildi...   
12  What is highlighted as a key consideration whe...   
13  Who proposed the FTI design pattern according ...   
14  What are the benefits mentioned in relation to...   
15  What problem does the described architecture s...   
16  What is

Filter:   0%|          | 0/35 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25 [00:00<?, ? examples/s]

Filtered Preference dataset:
                                               prompt  \
0   What is the FTI architecture designed to address?   
1   What are the three critical steps that any ML ...   
2   What is a significant aspect of the design fle...   
3   What are some responsibilities of an ML or MLO...   
4   What components are necessary for a mature ML ...   
5   How can all components of an ML system be conn...   
6   What is the typical architecture present in mo...   
7   What is the first step in building scalable ML...   
8   What is the FTI architecture used for in ML sy...   
9   What are the initial steps discussed in buildi...   
10  What is highlighted as a key consideration whe...   
11  What are the benefits mentioned in relation to...   
12  What problem does the described architecture s...   
13  What issue arises from building a monolithic b...   
14  What is required to make predictions in a real...   
15  What potential errors can arise from the descr...   
16